# A Demonstration of PCA

This notebook presents the inner working of Principal Component Analysis using a sample dataset. We will first build the theory the hard way , by avoiding the standard libraries for PCA. This will help us to understand , what exactly happens under the hood.

We start with the usual library imports and loading our sample dataset.

In [37]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import random

df=pd.read_csv('PCA Demo.csv')
df

,x1,x2,x3,x4,x5,x6,x7,x8,x9
0,90.200,10.000,41.917,44.900,1610,9.440,56.200,5.820,553
1,16.600,28.000,267.895,48.600,9930,4.490,76.300,1.650,4090
2,27.300,38.400,185.982,31.400,12900,16.100,76.500,2.890,4460
3,119.000,62.300,100.605,42.900,5900,22.400,60.100,6.160,3530
4,10.300,45.500,735.660,58.900,19100,1.440,76.800,2.130,12200
...,...,...,...,...,...,...,...,...,...
133,29.200,46.600,155.925,52.700,2950,2.620,63.000,3.500,2970
134,17.100,28.500,662.850,17.600,16500,45.900,75.400,2.470,13500
135,23.300,72.000,89.604,80.200,4490,12.100,73.100,1.950,1310
136,56.300,30.000,67.858,34.400,4480,23.600,67.500,4.670,1310


The dataset consists of 138 points of 9 dimensions or features. If we run a desribe method, we can see, there is a large variation in the mean values of the features. 

In [38]:
pd.options.display.float_format = "{:,.3f}".format
df.describe()

,x1,x2,x3,x4,x5,x6,x7,x8,x9
count,138.000,138.000,138.000,138.000,138.000,138.000,138.000,138.000,138.000
mean,45.274,38.256,372.444,47.023,"10,397.558",8.825,68.457,3.188,"5,722.565"
std,41.050,22.075,446.095,21.288,"9,503.147",11.192,8.355,1.554,"6,166.447"
min,3.200,0.109,12.821,0.066,609.000,-4.210,32.100,1.230,231.000
25%,14.525,22.800,57.229,31.650,"2,667.500",2.595,62.350,1.920,"1,177.500"
50%,27.700,34.000,195.480,45.100,"7,765.000",6.225,70.400,2.645,"3,490.000"
75%,64.275,50.425,503.782,58.900,"15,400.000",12.100,74.700,4.547,"8,170.000"
max,208.000,153.000,"2,209.200",154.000,"45,400.000",104.000,80.400,7.490,"30,800.000"


## Variance and Scaling

If we project these 9-dimensional points to another basis, there will be some loss of information. We measure this information based on the variance of each of the feature. Note , how each variance is just the square of the standard deviation of each feature we obtained above.

In [40]:
vrnc=[round(df[col].var(),2) for col in df.columns]
print("Variance of each Column")
print(vrnc)

Variance of each Column
[1685.14, 487.29, 199000.94, 453.19, 90309805.24, 125.27, 69.8, 2.42, 38025068.69]


Now we ask, how much does each of the variance contribute to the total variance in the dataset. We notice that X5 and X9 contribue to almost 99% of the total variance. 

In [4]:
total=0

for v in range(0, len(vrnc)):
    total = total + vrnc[v]

    vrnc_pcnt=[round(100*p/total,2) for p in vrnc]
print("Percentage Variance of each column \n",vrnc_pcnt)

Percentage Variance of each column 
 [0.0, 0.0, 0.15, 0.0, 70.26, 0.0, 0.0, 0.0, 29.58]


However X1 and X5 are also the biggest fetaures , in terms of magnitude. They might well overwhelm the other features. To mitigate , this we mean-normalize each feature by using __StandardScaler__. This effectively reduces the mean of each feature to 0 and standard deviation to 1. We call our scaled dataset as __X__.

In [5]:
scaler = StandardScaler()
X =pd.DataFrame(scaler.fit_transform(df),columns= df.columns)
X.describe()

,x1,x2,x3,x4,x5,x6,x7,x8,x9
count,138.000,138.000,138.000,138.000,138.000,138.000,138.000,138.000,138.000
mean,0.000,-0.000,-0.000,-0.000,0.000,-0.000,-0.000,0.000,0.000
std,1.004,1.004,1.004,1.004,1.004,1.004,1.004,1.004,1.004
min,-1.029,-1.734,-0.809,-2.214,-1.034,-1.169,-4.368,-1.264,-0.894
25%,-0.752,-0.703,-0.709,-0.725,-0.816,-0.559,-0.734,-0.819,-0.740
50%,-0.430,-0.193,-0.398,-0.091,-0.278,-0.233,0.233,-0.350,-0.363
75%,0.465,0.553,0.295,0.560,0.528,0.294,0.750,0.878,0.398
max,3.978,5.217,4.132,5.043,3.697,8.535,1.435,2.778,4.082


If we look at the contribution of the variance of ech feature, we notice that each of them are equal.

In [6]:
vrnc=[X[col].var() for col in X.columns]
total=0
for ele in range(0, len(vrnc)):
    total = total + vrnc[ele]

vrnc_pcnt=[round(100*p/total,2) for p in vrnc]
print("Percentage Variance of each column \n",vrnc_pcnt)

Percentage Variance of each column 
 [11.11, 11.11, 11.11, 11.11, 11.11, 11.11, 11.11, 11.11, 11.11]


## Building the Covariance Matrix

The basic building block of PCA is the covariance matrix. Recall, the covariance between any two features is given by

$\displaystyle cov (x , y) = \frac{1}{m} \sum_{j=1}^{m} (x_j-\bar{x})(y_j-\bar{y})$

Since , we have scaled every single feature, their mean has reduced to zero. Thus the covariance simply becomes

$\displaystyle cov (x , y) = \frac{1}{m} \sum_{j=1}^{m} x_j y_j$

However, if we look closely , the above equation is same multiplying __$X$__ with its transpose __$X^T$__ and dividing by m (the number of points).

$\displaystyle \frac{1}{m} X.X^T$

This is why , scaling each feature is an important step before we actually perform PCA.

In [7]:
X.cov()

,x1,x2,x3,x4,x5,x6,x7,x8,x9
x1,1.007,-0.343,-0.530,-0.174,-0.580,0.230,-0.880,0.836,-0.545
x2,-0.343,1.007,0.427,0.638,0.551,-0.099,0.318,-0.338,0.499
x3,-0.530,0.427,1.007,0.209,0.781,-0.193,0.554,-0.549,0.944
x4,-0.174,0.638,0.209,1.007,0.120,-0.312,0.079,-0.198,0.165
x5,-0.580,0.551,0.781,0.120,1.007,-0.056,0.597,-0.574,0.922
x6,0.230,-0.099,-0.193,-0.312,-0.056,1.007,-0.147,0.266,-0.123
x7,-0.880,0.318,0.554,0.079,0.597,-0.147,1.007,-0.735,0.573
x8,0.836,-0.338,-0.549,-0.198,-0.574,0.266,-0.735,1.007,-0.545
x9,-0.545,0.499,0.944,0.165,0.922,-0.123,0.573,-0.545,1.007


Let us multiply __$X$__ with its transpose __$X^T$__ and divide by m. Indeed !! We get almost the same result.

In [8]:
X1=X.to_numpy()
pd.DataFrame((X1.T@X1)/138,columns=X.columns)

,x1,x2,x3,x4,x5,x6,x7,x8,x9
0,1.000,-0.341,-0.527,-0.173,-0.576,0.229,-0.873,0.830,-0.541
1,-0.341,1.000,0.424,0.634,0.547,-0.099,0.316,-0.336,0.496
2,-0.527,0.424,1.000,0.207,0.775,-0.191,0.550,-0.545,0.937
3,-0.173,0.634,0.207,1.000,0.119,-0.310,0.078,-0.197,0.164
4,-0.576,0.547,0.775,0.119,1.000,-0.055,0.592,-0.570,0.915
5,0.229,-0.099,-0.191,-0.310,-0.055,1.000,-0.146,0.264,-0.122
6,-0.873,0.316,0.550,0.078,0.592,-0.146,1.000,-0.729,0.568
7,0.830,-0.336,-0.545,-0.197,-0.570,0.264,-0.729,1.000,-0.541
8,-0.541,0.496,0.937,0.164,0.915,-0.122,0.568,-0.541,1.000


We will represent the Covariance matrix with a $\Sigma$ symbol. This is a simple step to convert the dataframe __df_scaled__ to a numpy array. This is a square matrix of 9x9 dimension. The diagonal values are simply the variance of each variable. Note they are all equal to ~1.

In [9]:
sigma=X.cov().to_numpy()
sigma.shape

(9, 9)

## Singular Value Decomposition(SVD)
We will now perform SVD of $\Sigma$

$\displaystyle[U,S,U^{-1}]=svd(\Sigma)$

- U is a 9x9 matrix consisting of m eigenvectors of Σ and is the new basis we are looking for.
- S is a diagonal matrix consisting of the m eigen-values of Σ.

In [10]:
U,S,Ui= np.linalg.svd(sigma, full_matrices=True)

To project X to the new basis __U__ , we just multiply __X__ with __U__. Calling the projected data as __Xp__.

We will investigate the following properties of U and S.

- The eigenvalues of Σ represent the variance of the column of __Xp__.
- The variance of each column in __Xp__ appear in descending order.
- The eigenvectors in the matrix U are orthogonal to each other and have a magnitude of 1. This means their dot product will be zero and __norm__ will be 1.
- The columns in the projected dataset __Xp__ have zero covariance between any two columns.

In [33]:
#X=X.to_numpy()
Xp=pd.DataFrame(X@U)
print("Variance of each column of Xp")
print([round(Xp[i].var(),5) for i in range(9)])

print("Eigenvalues in S")
print([round(ev,5) for ev in S])

Variance of each column of Xp
[4.80909, 1.43238, 1.17451, 0.8125, 0.3153, 0.26171, 0.14407, 0.09722, 0.01892]
Eigenvalues in S
[4.80909, 1.43238, 1.17451, 0.8125, 0.3153, 0.26171, 0.14407, 0.09722, 0.01892]


If we add up the variances of the columns of __Xp__ and expressed each variance as a percent, we'll see that the columns arein the descending order of variance.

In [34]:
vrnc=[Xp[col].var() for col in Xp.columns]
total=0
for ele in range(0, len(vrnc)):
    total = total + vrnc[ele]

vrnc_pcnt=[round(100*p/total,2) for p in vrnc]
print("Percentage Variance of each column \n",vrnc_pcnt)

Percentage Variance of each column 
 [53.05, 15.8, 12.96, 8.96, 3.48, 2.89, 1.59, 1.07, 0.21]


In [36]:
np.array([Xp[col].var() for col in Xp.columns]).sum()

9.065693430656934

Note , that the first 6 values add up to ~97%. Next we demonstrate that any pair of vectors in U are orthogonal to each other. We generate a pair of random numbers between 0 to 8 , to point at any of the eigenvectors in __U__ and then multiply them. We also calculate their magnitudes.

In [46]:
rnd_vectors = random.sample(range(0, 9), 2)
print("Choosing Eignevectors",rnd_vectors)

u1=U[::,rnd_vectors[0]]
u2=U[::,rnd_vectors[1]]

print("Dot product of the two eigenvectors",round(np.dot(u1,u2),5))

print("Magnitudes of the two eigenvectors",round(np.linalg.norm(u1),3),round(np.linalg.norm(u2),3))

Choosing Eignevectors [4, 7]
Dot product of the two eigenvectors -0.0
Magnitudes of the two eigenvectors 1.0 1.0


Let us take the first eigenvector among the two and multiply it with sigma. We should get the same result , when we multiply the eigenvector with corresponding eigenvalue.

In [52]:
print(sigma@u1)
print(u1*S[rnd_vectors[0]])

[ 0.00173429  0.17040407 -0.13862499 -0.15648256  0.10404549 -0.09937067
  0.03318725  0.0634995  -0.0289193 ]
[ 0.00173429  0.17040407 -0.13862499 -0.15648256  0.10404549 -0.09937067
  0.03318725  0.0634995  -0.0289193 ]


In [53]:
print("Dot product of the two eigenvectors",round(np.dot(u1,u2),5))
print("Magnitudes of the two eigenvectors",round(np.linalg.norm(u1),3),round(np.linalg.norm(u2),3))

Dot product of the two eigenvectors -0.0
Magnitudes of the two eigenvectors 1.0 1.0


Next we show that the covariance matrix of __Xp__ is a diagonal matrix.

In [158]:
Xp.cov()

,0,1,2,3,4,5,6,7,8
0,4.809,-0.000,-0.000,-0.000,0.000,-0.000,-0.000,-0.000,0.000
1,-0.000,1.432,-0.000,0.000,0.000,0.000,0.000,-0.000,0.000
2,-0.000,-0.000,1.175,-0.000,0.000,-0.000,0.000,0.000,-0.000
3,-0.000,0.000,-0.000,0.812,0.000,-0.000,-0.000,0.000,0.000
4,0.000,0.000,0.000,0.000,0.315,0.000,0.000,-0.000,-0.000
5,-0.000,0.000,-0.000,-0.000,0.000,0.262,-0.000,0.000,-0.000
6,-0.000,0.000,0.000,-0.000,0.000,-0.000,0.144,-0.000,-0.000
7,-0.000,-0.000,0.000,0.000,-0.000,0.000,-0.000,0.097,0.000
8,0.000,0.000,-0.000,0.000,-0.000,-0.000,-0.000,0.000,0.019


## Using the Scikit Library

We will now use the scikit learn library to acheive a transformed dataset $X_{pca}$. This will be exactly same as $X_p$ we found above. This is a simpler and more direct approach to arrive at the projected dataset along its principal components.

In [61]:
from sklearn.decomposition import PCA
pca = PCA(svd_solver='randomized', random_state=100)
Xpca=pca.fit_transform(X)

In [62]:
pd.DataFrame(Xpca)

,0,1,2,3,4,5,6,7,8
0,-2.962,0.226,0.293,-0.749,0.371,-0.337,0.460,-0.336,-0.055
1,0.690,-0.465,-1.365,0.379,0.160,0.070,0.167,0.332,-0.022
2,0.264,-1.079,-0.174,0.875,-0.515,-0.259,0.044,0.231,-0.049
3,-2.200,0.849,2.146,0.430,-0.739,-0.448,-0.489,0.186,0.103
4,2.307,0.107,-0.260,-0.320,0.081,-0.265,0.281,0.107,0.122
...,...,...,...,...,...,...,...,...,...
133,-0.633,0.828,-0.651,0.100,-0.213,0.113,-0.250,-0.730,0.179
134,1.018,-2.928,1.981,1.610,0.799,-0.008,-0.112,-0.149,0.452
135,0.555,1.613,-0.720,1.954,0.005,0.115,-0.382,0.165,0.048
136,-1.655,-0.727,0.508,0.909,-0.018,-0.497,-0.045,-0.156,-0.045


We can deduce the eigenvalues. It is same as the __S__ matrix we obtained using the numpy linalg method. 

In [68]:
print('pca.explained_variance_ratio_: ',pca.explained_variance_.round(3))

pca.explained_variance_ratio_:  [4.809 1.432 1.175 0.812 0.315 0.262 0.144 0.097 0.019]


## Conclusions

We are now in a position to draw conclusion from all the math we did so far. 
- To perform dimensionality reduction on a n-dimensional dataset __X__, we need to normalize it first and then deduce its covariance matrix $\Sigma$.
- The SVD of the $\Sigma$ gives us a nxn matrix __U__ consisting of n eigenvectors and a nxn diagonal matrix __S__ consisting of n eigenvalues.
- The eigenvectors have unit norm and orthogonal to each other.
- To project X on the new basis __U__ , we simply need to multiply __X__ and __U__ to get __$X_p$__.
- The columns of __$X_p$__ are in descending order of variance.The variance of each column are the eigenvalues that appear in __S__.
- We can drop some of columns in the preojected dataset, based on how much variance we want to retain.